In [21]:
import numpy as np
from scipy import linalg
from numpy.linalg import inv


In [3]:
#state etc intialized here
x_ =  np.array([[5.7441],
                [1.3800],
                [2.2049],
                [0.5015],
                [0.3528]])

P_ = np.array([[0.0043,   -0.0013,    0.0030,   -0.0022,   -0.0020],
               [-0.0013,    0.0077,    0.0011,    0.0071,    0.0060],
               [0.0030,    0.0011,    0.0054,    0.0007,    0.0008],
               [-0.0022,    0.0071,    0.0007,    0.0098,    0.0100],
               [-0.0020,    0.0060,    0.0008,    0.0100,    0.0123]])
n_x_ = 5

n_aug_ = 7
lambda_ = 3 - n_aug_
std_a = 0.2
std_yawdd = 0.2
n_z = 3
std_radr = 0.3
std_radphi = 0.0175
std_radrd = 0.1



In [4]:
#predict
x_aug = np.zeros((n_aug_,1))
x_aug[:n_x_,:]=x_

P_aug = np.zeros((n_aug_, n_aug_))
P_aug[:n_x_,:n_x_] = P_
P_aug[-2,-2] = std_a * std_a
P_aug[-1,-1] = std_yawdd * std_yawdd

Xsig = np.zeros((n_aug_, 2*n_aug_ + 1))
#A = linalg.sqrtm(P_)
A =  np.array([[0.0655744,0,0,0,0,0,0],
               [-0.0198248,0.0854809,0,0,0,0,0],
               [0.0457496,0.0234787,0.052495,0,0,0,0],
               [-0.0335497,0.0752787,0.00890448,0.0541133,0,0,0],
               [-0.0304997,0.0631176,0.0135904,0.0758467,0.0380593,0,0],
               [0,0,0,0,0,0.2,0],
               [0,0,0,0,0,0,0.2]])
Xsig[:,0] =  x_aug.ravel()
Xsig[:,1:n_aug_+1] = A * (lambda_ + n_aug_)**0.5 + x_aug.reshape((n_aug_,-1))
Xsig[:,n_aug_+1:2*n_aug_+1] = - A * (lambda_ + n_aug_)**0.5 + x_aug
delta_t = 0.1
Xsig_pred = np.zeros((n_x_, 2 *n_aug_ + 1))

           
for i in range(len(Xsig[0])):
    v = Xsig[2,i]
    yaw = Xsig[3,i]
    yaw_dot = Xsig[4,i]
    v_a = Xsig[5,i]
    v_yaw_ddot = Xsig[6,i]
    
    if yaw_dot != 0:
        Xsig_pred[0,i] = Xsig[0,i] + v / yaw_dot * (np.sin(yaw + yaw_dot * delta_t) - np.sin(yaw)) + 1/2 * delta_t * delta_t * np.cos(yaw) * v_a
        Xsig_pred[1,i] = Xsig[1,i] + v / yaw_dot * (-np.cos(yaw + yaw_dot * delta_t) + np.cos(yaw)) + 1/2 * delta_t * delta_t * np.sin(yaw) * v_a
    else:
        Xsig_pred[0,i] = Xsig[0,i] + v * np.cos(yaw) * delta_t  + 1. / 2. * delta_t * delta_t * np.cos(yaw) * v_a                       
        Xsig_pred[1,i] = Xsig[1,i] + v * np.sin(yaw) * delta_t  + 1. / 2. * delta_t * delta_t * np.sin(yaw) * v_a        
    Xsig_pred[2,i] = Xsig[2,i] + delta_t *  v_a
    Xsig_pred[3,i] = Xsig[3,i] + yaw_dot * delta_t + 1/2 * delta_t * delta_t * v_yaw_ddot
    Xsig_pred[4,i] = Xsig[4,i] +  delta_t * v_yaw_ddot

#overwriting Xsig_pred as per example
Xsig_pred = np.array([
         [5.9374,  6.0640,   5.925,  5.9436,  5.9266,  5.9374,  5.9389,  5.9374,  5.8106,  5.9457,  5.9310,  5.9465,  5.9374,  5.9359,  5.93744],
           [1.48,  1.4436,   1.660,  1.4934,  1.5036,    1.48,  1.4868,    1.48,  1.5271,  1.3104,  1.4787,  1.4674,    1.48,  1.4851,    1.486],
          [2.204,  2.2841,  2.2455,  2.2958,   2.204,   2.204,  2.2395,   2.204,  2.1256,  2.1642,  2.1139,   2.204,   2.204,  2.1702,   2.2049],
         [0.5367, 0.47338, 0.67809, 0.55455, 0.64364, 0.54337,  0.5367, 0.53851, 0.60017, 0.39546, 0.51900, 0.42991, 0.530188,  0.5367, 0.535048],
          [0.352, 0.29997, 0.46212, 0.37633,  0.4841, 0.41872,   0.352, 0.38744, 0.40562, 0.24347, 0.32926,  0.2214, 0.28687,   0.352, 0.318159]
])

    
weights = np.zeros((2 * n_aug_ + 1))
weights[0] = lambda_ / (lambda_ + n_aug_)
weights[1:] = 1/(2 * (lambda_ + n_aug_))
x_pred = np.sum(weights * Xsig_pred, axis=1)
D = Xsig_pred-x_pred.reshape((n_x_,1))
E = D * weights
P_pred = np.dot(E , D.transpose())
P_pred

array([[ 0.00543425, -0.0024053 ,  0.00341576, -0.00348196, -0.00299378],
       [-0.0024053 ,  0.010845  ,  0.0014923 ,  0.00980182,  0.00791091],
       [ 0.00341576,  0.0014923 ,  0.00580129,  0.00077863,  0.00079297],
       [-0.00348196,  0.00980182,  0.00077863,  0.01192378,  0.01124909],
       [-0.00299378,  0.00791091,  0.00079297,  0.01124909,  0.01269717]])

In [35]:
#update radar
Zsig = np.zeros((n_z, 2* n_aug_ + 1))
for i in range(2 * n_aug_ + 1):
    Zsig[0,i] = (Xsig_pred[0,i] * Xsig_pred[0,i] + Xsig_pred[1,i] * Xsig_pred[1,i])** 0.5
    Zsig[1,i] = np.arctan(Xsig_pred[1,i] / Xsig_pred[0,i])
    Zsig[2,i] = (Xsig_pred[0,i] * np.cos(Xsig_pred[3,i])*Xsig_pred[2,i] + 
                    Xsig_pred[1,i] * np.sin(Xsig_pred[3,i])*Xsig_pred[2,i]) / Zsig[0,i]

C = Zsig * weights
z_pred = np.sum(C, axis=1)

D = Zsig - z_pred.reshape(n_z,1)
E = D * weights
R = np.zeros((n_z, n_z))
R[0,0] = std_radr * std_radr
R[1,1] = std_radphi * std_radphi
R[2,2] = std_radrd * std_radrd        
S = np.dot(E, D.transpose()) + R

z = np.array([[5.9214],
              [0.2187],
              [2.0062]]) 
M1 = Xsig_pred-x_pred.reshape((n_x_,1))
M2 = M1 * weights
M3 = Zsig - z_pred.reshape(n_z,1)
Tc = np.dot(M2, M3.transpose())

K = np.dot(Tc, inv(S))

x_upd = x_pred + np.dot(K, (z - z_pred.reshape(n_z,1)).ravel())
P_upd = P_pred -1 * np.dot(K,np.dot(S,K.transpose()))
P_upd

array([[ 0.00361563, -0.00035299,  0.00208269, -0.00093334, -0.00071443],
       [-0.00035299,  0.00540046,  0.00157554,  0.00454782,  0.00358316],
       [ 0.00208269,  0.00157554,  0.00410593,  0.00160861,  0.00172192],
       [-0.00093334,  0.00454782,  0.00160861,  0.00652055,  0.0066884 ],
       [-0.00071443,  0.00358316,  0.00172192,  0.0066884 ,  0.00881277]])

In [5]:
#update lidar